In [1]:
import pandas as pd 

In [2]:
sample_data = pd.read_csv('puzzle1(2).csv') # csv file reading

In [3]:
sample_data

,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,0,0,9,0,6,5,4,3,0
1,0,0,7,0,0,0,8,0,0
2,6,0,0,1,0,8,0,2,0
3,0,0,3,0,9,0,0,0,2
4,5,0,1,4,0,3,9,6,0
5,8,0,4,0,0,0,1,0,0
6,0,3,0,5,0,9,0,0,7
7,0,5,6,0,8,0,0,0,0
8,0,7,0,2,4,0,0,9,0


In [4]:
notation = []

In [5]:
for i in range(9):
    for j in range(9):
        if sample_data.iloc[i,j] == 0:
            sample_data.iloc[i,j] = 'x'
        notation.append(sample_data.iloc[i,j])

In [6]:
sample_data

,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,x,x,9,x,6,5,4,3,x
1,x,x,7,x,x,x,8,x,x
2,6,x,x,1,x,8,x,2,x
3,x,x,3,x,9,x,x,x,2
4,5,x,1,4,x,3,9,6,x
5,8,x,4,x,x,x,1,x,x
6,x,3,x,5,x,9,x,x,7
7,x,5,6,x,8,x,x,x,x
8,x,7,x,2,4,x,x,9,x


In [7]:
start = ''.join(str(v) for v in notation) # all the entries in the csv file are being converted in to string

In [8]:
start

'xx9x6543xxx7xxx8xx6xx1x8x2xxx3x9xxx25x14x396x8x4xxx1xxx3x5x9xx7x56x8xxxxx7x24xx9x'

In [9]:
def cartesian_product(x,y):
    
    return [a+b for a in x for b in y] 
# takes two iterable values and return the cartesian product in a list


In [10]:
# displays the game board

def display_game_board(values):
    
    print('')
    
    rows = 'ABCDEFGHI'
    cols = '123456789'
    boxes = cartesian_product(rows, cols)
    
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                    for c in cols))
        if r in 'CF': print(line)
    return

In [11]:
# elimination function eliminates the possible values according to the rules to get a simplified version of the puzzle.
def eliminate(Grid):
    for k,v in Grid.items():
        if len(v) != 1: #  checks if the box needs elimination
            peers = peer_dict[k]    # takes all the peers
            peer_values = set([Grid[p] for p in peers if len(Grid[p]) == 1])
            Grid[k] = ''.join(set(Grid[k]) - peer_values)
    return Grid

In [12]:
# checks the places the number if the there is only one choice to place it 
def choice(Grid):
    for unit in unit_list:
        for num in '123456789':
            num_places = [box for box in unit if num in Grid[box]]
            if len(num_places) == 1:
                Grid[num_places[0]] = num
    return Grid

In [13]:
# Kindly check the documentation to know about naked_pairs

def naked_pairs(Grid):
    for unit in unit_list:
        
        # slice the Grid to contain only the boxes in the unit
        values = dict([[box, ''.join(sorted(Grid[box]))] for box in unit])
        
        # find all the items with 2-digit values
        double_digits = dict([[box, values[box]] for box in values if len(values[box])==2])
        
        # check if any of those 2-digit values occur exactly twice
        double_digits_occuring_twice = dict([[box, val] for box, val in double_digits.items() if list(double_digits.values()).count(val)==2])
        
        if len(double_digits_occuring_twice.items()) != 0:
            # reverse the dictionary to get the key-pairs easily
            reverse_dict = {}
            for k, v in double_digits_occuring_twice.items():
                reverse_dict.setdefault(v, []).append(k)

            # it is a list of 2 items(keys | boxes) only
            naked_pairs = list(reverse_dict.items())[0][1]

            # remove the naked_pairs digits from other boxes in the unit
            for k,v in values.items():
                if (k not in naked_pairs) and (len(v) > 1):
                    values[k] = ''.join(set(values[k]) - set(values[naked_pairs[0]]))
    
        # replace the values in Grid with the updated values
        for k,v in values.items():
            Grid[k] = v
    
    return Grid


In [14]:
def run(Grid):
    stuck = False
    while not stuck:
        # Check how many boxes have a fixed value
        previous_solved = len([box for box in Grid.keys() if len(Grid[box]) == 1])
        
        
        Grid = eliminate(Grid)
        
        
        Grid = choice(Grid)
        
        
        Grid = naked_pairs(Grid)
        
        # Check how many boxes have a value, to compare
        post_solved_values = len([box for box in Grid.keys() if len(Grid[box]) == 1])
        
        # If no new values were added, stop the loop.
        stuck = previous_solved == post_solved_values
        
        
        # if the current sudoku board is cannot be solved then return False
        if len([box for box in Grid.keys() if len(Grid[box]) == 0]):
            return False 
    return Grid


In [15]:

def search(Grid):
    Grid = run(Grid)
    
    if Grid is False:
        return False
    
    if all(len(v) == 1 for k,v in Grid.items()): 
        return Grid
    
    # Choose one of the unfilled squares with the fewest possibilities
    length,k = min((len(val), key) for key,val in Grid.items() if len(val) > 1)
    # print(k, length)
    
    # Now use recurrence to solve each one of the resulting sudoku
    for digit in Grid[k]:
        new_sudoku = dict(list(Grid.items()))
        new_sudoku[k] = digit
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [16]:
if __name__ == '__main__':
    
    
    
    
    rows = 'ABCDEFGHI'
    cols = '123456789'
    boxes = cartesian_product(rows, cols)

    row_units = [cartesian_product(r, cols) for r in rows]
    col_units = [cartesian_product(rows, c) for c in cols]
    box_units = [cartesian_product(r,c) 
                    for r in ['ABC', 'DEF', 'GHI'] 
                    for c in ['123','456','789']]

    unit_list = row_units + col_units + box_units

    # each box(key) with its units(value)
    unit_dict = dict((box, [unit for unit in unit_list if box in unit]) for box in boxes)
    
    # each box with its peers
    peer_dict = dict((box, set(sum(unit_dict[box], [])) - set([box])) for box in boxes)

    # start string converted to dictionary
    assert len(start) == 81
    Grid = dict(zip(boxes, start))

    # replacing the x with '123456789' (possible values in the box)
    for k,v in Grid.items():
        if v == 'x':
            Grid[k] = '123456789'
            
    solved_grid = search(Grid)
    
    display_game_board(solved_grid)


2 8 9 |7 6 5 |4 3 1 
3 1 7 |9 2 4 |8 5 6 
6 4 5 |1 3 8 |7 2 9 
------+------+------
7 6 3 |8 9 1 |5 4 2 
5 2 1 |4 7 3 |9 6 8 
8 9 4 |6 5 2 |1 7 3 
------+------+------
4 3 2 |5 1 9 |6 8 7 
9 5 6 |3 8 7 |2 1 4 
1 7 8 |2 4 6 |3 9 5 


In [17]:
#new_grid = eliminate(Grid)

In [18]:
#display_game_board(new_grid)

In [ ]:
#display_game_board(naked_dict)

In [ ]:
#display_game_board(Grid)